In [22]:
import argparse
from tqdm import trange
# from torchtext.vocab import Vectors, GloVe, CharNGram, FastText
from nltk.tokenize.treebank import TreebankWordDetokenizer
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
from torchtext import data as torchtext_data
from torchtext import datasets

import jittor as jt
import gpt2
import time

import transformers

In [23]:
class myDataset(jt.dataset.Dataset):
    def __init__(self, x, y):
        super().__init__()
        self.x = x #2d
        self.y = jt.array(y) #1d
        lengths = [len(seq) for seq in x]
        padding = jt.zeros(len(lengths), max(lengths))
        for i, seq in enumerate(x):
            padding[i,:lengths[i]] = seq[:lengths[i]]
        self.x = padding
    
    def __getitem__(self, k):
        return self.x[k], self.y[k]
    
    def __len__(self):
        return len(self.y)
tokenizer = transformers.GPT2Tokenizer.from_pretrained('gpt2-medium')
# model = transformers.GPT2LMHeadModel.from_pretrained('gpt2-medium')

In [24]:
class ClassificationHead(jt.nn.Module):
    def __init__(self, class_size=5, embed_size=2048):
        super().__init__()
        self.class_size = class_size
        self.embed_size = embed_size
        self.mlp = jt.nn.Linear(embed_size, class_size)
    def execute(self, hidden_state):
        lm_logits = self.mlp(hidden_state)
        return lm_logits

class Discriminator2mean(jt.nn.Module):
    def __init__(self, class_size=5, embed_size=1024, head=None):
        super().__init__()
        if head == None:
            self.classifierhead = ClassificationHead(class_size=class_size, embed_size=embed_size)
        else:
            self.classifierhead = head
        self.model = transformers.GPT2LMHeadModel.from_pretrained('gpt2-medium')
        self.embed_size = embed_size
    
    def get_classifier(self):
        return self.classifierhead

    def get_classifier_param(self):
        return self.classifierhead.parameters()

    def execute(self, x):
        mask_src = 1 - x.equal(0).unsqueeze(1).detach()
        mask_src = mask_src.repeat(1, self.embed_size, 1) #batch_size, 1024, length (repeat each sentence for 1024 times)
        mask_src = torch.tensor(mask_src.tolist())
        x = torch.tensor(x.tolist(),dtype=torch.long)
        output_dict = self.model(x, output_hidden_states=True)
        hidden = output_dict.hidden_states[-1]

        hidden = hidden.permute(0, 2, 1)
        hidden = hidden * mask_src  # / torch.sum(mask_src, dim=-1).unsqueeze(2).repeat(1, 1, batch_length)
        #
        hidden = hidden.permute(0, 2, 1)
        x =  torch.sum(hidden, dim=1)/(torch.sum(mask_src, dim=-1) + 1e-10)
        
        x = jt.array(x.tolist(), dtype=jt.int64)
        x = self.classifierhead(x)
        x = jt.nn.log_softmax(x, dim=-1)
        return x

In [25]:
jt.flags.use_cuda = jt.has_cuda

parser = argparse.ArgumentParser(description='Train a discriminator on top of GPT-2 representations')
parser.add_argument('--batch_size', type=int, default=64, metavar='N',
                    help='input batch size for training (default: 64)')
parser.add_argument('--log_interval', type=int, default=10, metavar='N',
                    help='how many batches to wait before logging training status')
parser.add_argument('--epochs', type=int, default=10, metavar='N',
                    help='Number of training epochs')
parser.add_argument('--save_path', type=str, default='', help='whether to save the model')
parser.add_argument('--load_path', type=str, default='')
parser.add_argument('--dataset_label', type=str, default='SST',choices=('SST', 'clickbait', 'toxic'))
args = parser.parse_args(args=[])

In [26]:
if args.dataset_label == 'SST':
    text = torchtext_data.Field()
    label = torchtext_data.Field(sequential=False)
    train_data, val_data, test_data = datasets.SST.splits(text, label, fine_grained=True, train_subtrees=True,
                                                            # filter_pred=lambda ex: ex.label != 'neutral'
                                                            )
    d = {"positive": 0, "negative": 1, "very positive": 2, "very negative": 3, "neutral": 4}
    x = []
    y = []
    for i in range(len(train_data)):
        seq = TreebankWordDetokenizer().detokenize(vars(train_data[i])["text"])
        seq = tokenizer.encode(seq)
        x.append(seq)
        y.append(d[vars(train_data[i])["label"]])
    train_dataset = myDataset(x,y).set_attrs(batch_size=args.batch_size,shuffle=True)

    test_x = []
    test_y = []
    for i in range(len(test_data)):
        seq = TreebankWordDetokenizer().detokenize(vars(test_data[i])["text"])
        seq = tokenizer.encode(seq)
        seq = [50256] + seq
        #seq = torch.tensor([50256] + seq, device=device, dtype=torch.long)
        test_x.append(seq)
        test_y.append(d[vars(test_data[i])["label"]])
    test_dataset = myDataset(test_x, test_y).set_attrs(batch_size=args.batch_size,shuffle=True)
elif args.dataset_label == 'clickbait':
    pass
else:
    pass

In [27]:
def train_epoch(data_loader, discriminator:Discriminator2mean, args=None):
    optimizer = jt.optim.Adam(discriminator.get_classifier_param(), lr=0.0001)
    for epoch in range(args.epochs):
        for batch_idx, (data, target) in enumerate(data_loader):
            print('Epoch: {}, batch: {}'.format(epoch,batch_idx))
            start = time.time()
            data, target = data, target.reshape(-1) # data is 2-d list [batch_size, length(after padding)], target is 1-d list [batch_size]
            optimizer.zero_grad()
            output = discriminator(data)
            loss = jt.nn.nll_loss(output, target)
            optimizer.step(loss)
            print('batch time cost: {}'.format(time.time() - start))
            if batch_idx % args.log_interval == 0:
                print('Relu Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(data_loader),
                        batch_idx * len(data) / len(data_loader), loss.item()))
        head = discriminator.get_classifier()
        head.save(args.dataset_label+'-'+str(epoch)+'.pkl')

In [28]:
def test_epoch(data_loader, discriminator, args=None):
    discriminator.eval()
    test_loss = 0
    correct = 0
    with jt.no_grad():
        for data, target in data_loader:
            output = discriminator(data)
            test_loss += jt.nn.nll_loss(output, target.reshape(-1)).item()  # sum up batch loss
            pred,_ = output.argmax(dim=1, keepdims=True)  # get the index of the max log-probability
            correct += pred.equal(target.reshape(pred.shape)).sum().item()

    print('\nRelu Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(data_loader),
        100. * correct / len(data_loader)))

Relu Test set: Average loss: 57.9225, Accuracy: 545/2210 (25%)

In [29]:
model = Discriminator2mean()
train_epoch(train_dataset, model, args)

Epoch: 0, batch: 0


/var/folders/fg/906sj12s2650mts7g69skdwc0000gn/T/ipykernel_2850/4016484757.py:31: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  x = torch.tensor(x.tolist(),dtype=torch.long)


loss None
batch time cost: 11.179395198822021
Relu Train Epoch: 0 [0/318582 (0%)]	Loss: 6.557239
Epoch: 0, batch: 1
loss None
batch time cost: 5.505015850067139
Epoch: 0, batch: 2
loss None
batch time cost: 5.310577154159546
Epoch: 0, batch: 3
loss None
batch time cost: 5.27536416053772
Epoch: 0, batch: 4
loss None
batch time cost: 5.247416734695435
Epoch: 0, batch: 5
loss None
batch time cost: 8.259428977966309
Epoch: 0, batch: 6
loss None
batch time cost: 8.46963381767273
Epoch: 0, batch: 7
loss None
batch time cost: 8.646962881088257
Epoch: 0, batch: 8
loss None
batch time cost: 8.833514928817749
Epoch: 0, batch: 9
loss None
batch time cost: 5.352841138839722
Epoch: 0, batch: 10
loss None
batch time cost: 5.294504880905151
Relu Train Epoch: 0 [640/318582 (0%)]	Loss: 5.426012
Epoch: 0, batch: 11
loss None
batch time cost: 5.253703832626343
Epoch: 0, batch: 12
loss None
batch time cost: 5.6236412525177
Epoch: 0, batch: 13
loss None
batch time cost: 5.2580578327178955
Epoch: 0, batch: 

KeyboardInterrupt: 